## 实现第一个cnn

实现一个 0 - 9 的手写数字识别

In [11]:
import tensorflow as tf

## 准备数据

如第2章所示，我们使用Tensorflow和Keras助手加载常用的[MNIST](http://yann.lecun.com/exdb/mnist)续联和测试数据集。我们还对图像进行规范化（将像素值从“[0,255]”设置为“[0,1]”，并对其进行适当的重塑（因为Tensorflow将其存储为列向量）：

In [12]:
# 输出类别
num_classes = 10
# 图片长、宽、通道
img_rows, img_cols, img_ch = 28, 28, 1
# 输出形状
input_shape = (img_rows, img_cols, img_ch)

# 加载数据
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 归一化，将 0-255 => 0 - 1
x_train, x_test = x_train / 255.0, x_test / 255.0


In [13]:
print(x_train.shape)

(60000, 28, 28)


In [14]:
x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)


In [15]:
print(x_train.shape)


(60000, 28, 28, 1)


## 创建和训练 LeNet-5

我们已经展示了如何根据不同的需求，以不同的方式实现CNN。在本例中，我们将使用Keras API再次展示它使实现和使用神经网络变得多么简单。

### 实例化卷积层

在上一联系中，我们介绍了如何对图像执行卷积。然而，在神经网络中，我们希望卷积滤波器是***可训练的***，我们可能希望在结果中添加***偏差***，并应用***激活函数***。

因此，我们需要将卷积运算包装成一个“Layer”对象，类似于我们在第1章中实现的完全连接层是如何围绕矩阵运算构建的。

TensorFlow 2/Keras提供了自己的tf。克拉斯。我们可以扩展的`tf.keras.Layer`。下面我们将演示如何以这种方式定义简单的卷积层：


In [16]:
class SimpleConvolutionLayer(tf.keras.layers.Layer):

    def __init__(self, num_kernels=32, kernel_size=(3, 3), strides=(1, 1), use_bias=True):
        """
        初始化 layer.
        :param num_kernels:     卷积核数量
        :param kernel_size:     核尺寸 (H x W)
        :param strides:         垂直和水平的步长列表
        :param use_bias:        偏差b after covolution / 激活前
        """
        # First, we have to call the `Layer` super __init__(), as it initializes hidden mechanisms:
        super().__init__()
        # Then we assign the parameters:
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.strides = strides
        self.use_bias = use_bias

    def build(self, input_shape):
        """
        构建 layer, 根据输入形状初始化其参数.
        不过，第一次使用该层时，将在内部调用该函数，它也可以手动调用.
        :param input_shape: 输入图层将接收的形状(e.g. B x H x W x C)
        """
        #  获取 通道数量:
        num_input_channels = input_shape[-1]  # assuming shape format BHWC

        # 重新调整核的形状
        kernels_shape = (*self.kernel_size,
                         num_input_channels, self.num_kernels)

        # 本案例中, 我们使用从Glorot分布中选取的值初始化过滤器:
        glorot_uni_initializer = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(name='kernels',
                                       shape=kernels_shape,
                                       initializer=glorot_uni_initializer,
                                       trainable=True)  # 可训练的变量.

        if self.use_bias:  # 如果需要偏移量，则也需要初始化:
            self.bias = self.add_weight(name='bias',
                                        shape=(self.num_kernels,),
                                        # e.g., using normal distribution.
                                        initializer='random_normal',
                                        trainable=True)

    def call(self, inputs):
        """
        调用层并对输入张量执行其操作
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        # 进行卷积操作:
        z = tf.nn.conv2d(inputs, self.kernels, strides=[
                         1, *self.strides, 1], padding='VALID')

        if self.use_bias:  # 如果设置偏移量:
            z = z + self.bias
        # 最终我们使用激活函数 (e.g. ReLU):
        return tf.nn.relu(z)

    def get_config(self):
        """
        辅助函数返回定义的层和参数信息.
        :return:        Dictionary containing the layer's configuration
        """
        return {'num_kernels': self.num_kernels,
                'kernel_size': self.kernel_size,
                'strides': self.strides,
                'use_bias': self.use_bias}


大多数TensorFlow数学运算（例如在 `tf.maths` 和 `tf.nn`）已经有了框架定义的导数。

因此，只要一个层由这样的操作组成，**我们就不必手动定义它的反向传播**。TensorFlow将自动覆盖这一点，这节省了很多精力！

因此，我们刚刚实现的卷积层是完全可操作的，并且可以在CNN中使用，我们将马上演示  

***注：**由于卷积层是CNN最基本的组成部分，TensorFlow显然提供了自己的`tf.keras.layers.Conv2D`类。
`tf.keras.layers`模块包含大量预先实现的标准层，我们建议尽可能使用（因为它们有更多高级接口和优化的操作）。
为了演示，在本笔记本的其余部分，我们仍将使用我们自己更简单的“SimpleRevolutionLayer”，同时使用其他Keras预定义层。

### 实现LeNet-5

是一个7层的CNN网络（2个卷积层，2个 max 池化层 ，3 个全连接层 + 1个 在 全连接层前整合 feature map的 展开层). 

以下，我们将 扩展 `tf.keras.Model`类，用于自定义

In [17]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [18]:
class LeNet5(Model):

    def __init__(self, num_classes):
        """
        初始化模型.
        :param num_classes:    要从中预测的类数
        """
        super(LeNet5, self).__init__()
        # # 我们实例化了组成LeNet-5的各个层:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, kernel_size=(
            5, 5), padding='same', activation='relu')
        self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        """
        调用层并对输入张量执行操作
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x)))  # dense layers
        return x


### CNN 经典 MNIST

现在我们可以实例化并编译数字分类模型。为了训练它完成这项任务，我们实例化了优化器（本例中是一个简单的 _SGD_ ）并定义了损失（ _分类交叉熵_）


In [19]:
model = LeNet5(num_classes)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])


我们的模型 继承了`tf.keras.Model` 具有所有功能。例如，我们可以调用  `model.summary()` 要打印其摘要：

In [20]:
# 再调用 `model.summary()` 之前 ，我们必须构建它.
# 它通常在第一次组建网络时自动完成,
# 根据网络中的样本推断出输入形状.
# 例如，下面的命令将构建网络(then use it for prediction):
_ = model.predict(x_test[:10])

# 但我们可以手动构建模型，提供批处理的
# input shape ourselves:
batched_input_shape = tf.TensorShape((None, *input_shape))
model.build(input_shape=batched_input_shape)

# Method to visualize the architecture of the network:
model.summary()


Model: "le_net5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  156       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  2416      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  48120     
_________________________________________________________________
dense_1 (Dense)              multiple                  10164     
_________________________________________________________________
dense_2 (Dense)              multiple                  850 

在启动培训之前，我们还实例化了一些Keras回调，即在培训期间的特定点（批量培训之前/之后、完整纪元之前/之后等）自动调用的实用函数，以便对其进行监控：

In [21]:
callbacks = [
    # 如果验证丢失（`val_loss`）在超过3个时期内停止改善，则回调以中断培训
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # 回调以将图表、损失和指标记录到TensorBoard中 (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]


( Tensorboard 回调允许我们监控训练. 我们可以使用命令行 `tensorboard --logdir=./logs`. 打开浏览器的 [`localhost:6006`](localhost:6006) 浏览相关信息

我们可以将所有内容传递给我们的模型来训练它

In [22]:
history = model.fit(x_train, y_train,
                    batch_size=32, epochs=80, validation_data=(x_test, y_test),
                    verbose=2,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

# 1875不是样本个数，而是steps。
# steps跟batch_size有关，model.fit中的参数model.fit默认值是32，所以steps = 60000 / 32 = 1875。
# 可以自己设置batch_size，例如设置batch_size = 1，则steps = 60000。


Epoch 1/80
1875/1875 - 20s - loss: 0.4881 - accuracy: 0.8468 - val_loss: 0.1594 - val_accuracy: 0.9512
Epoch 2/80
1875/1875 - 18s - loss: 0.1305 - accuracy: 0.9604 - val_loss: 0.0882 - val_accuracy: 0.9729
Epoch 3/80
1875/1875 - 19s - loss: 0.0930 - accuracy: 0.9714 - val_loss: 0.0851 - val_accuracy: 0.9740
Epoch 4/80
1875/1875 - 18s - loss: 0.0754 - accuracy: 0.9762 - val_loss: 0.0633 - val_accuracy: 0.9808
Epoch 5/80
1875/1875 - 18s - loss: 0.0645 - accuracy: 0.9796 - val_loss: 0.0551 - val_accuracy: 0.9823
Epoch 6/80
1875/1875 - 17s - loss: 0.0566 - accuracy: 0.9823 - val_loss: 0.0473 - val_accuracy: 0.9841
Epoch 7/80
1875/1875 - 17s - loss: 0.0514 - accuracy: 0.9837 - val_loss: 0.0450 - val_accuracy: 0.9855
Epoch 8/80
1875/1875 - 17s - loss: 0.0458 - accuracy: 0.9858 - val_loss: 0.0407 - val_accuracy: 0.9862
Epoch 9/80
1875/1875 - 17s - loss: 0.0413 - accuracy: 0.9868 - val_loss: 0.0532 - val_accuracy: 0.9820
Epoch 10/80
1875/1875 - 17s - loss: 0.0381 - accuracy: 0.9881 - val_loss: